In [2]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import tweepy
import json
from timeit import default_timer as timer
from tweepy import OAuthHandler

In [4]:
## copy paste these
consumer_key=''
consumer_secret=''
access_token=''
access_token_secret=''

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

# I. Content

# 1. Gathering Data

In [6]:
screen_name='mindbodygreen' #user name
count=200    #max=200
page=0       #page 0 represent most recent tweets
tweets=api.user_timeline(screen_name=screen_name,count=count,page=0)

In [7]:
# scraping pages from 0 to 29
pages = np.arange(30)

txts = []
for page in pages: 
    tweets = api.user_timeline(screen_name=screen_name, count=count, page=page)
    for tweet in tweets: 
        txts.append(json.loads(json.dumps(tweet._json)))

len(txts)

3413

In [8]:
txts[3412]

{'created_at': 'Wed Nov 27 22:01:01 +0000 2019',
 'id': 1199810385127313408,
 'id_str': '1199810385127313408',
 'text': 'Your favorite, kid-approved side dish just got a low-carb makeover. 😋 https://t.co/qgMk7pxFSV',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [{'url': 'https://t.co/qgMk7pxFSV',
    'expanded_url': 'http://ow.ly/xfys30pVyHW',
    'display_url': 'ow.ly/xfys30pVyHW',
    'indices': [70, 93]}]},
 'source': '<a href="https://www.hootsuite.com" rel="nofollow">Hootsuite Inc.</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 17781673,
  'id_str': '17781673',
  'name': 'mindbodygreen',
  'screen_name': 'mindbodygreen',
  'location': 'Brooklyn, NY',
  'description': 'connecting soul & science. 💫',
  'url': 'http://t.co/xQ41BHxbiB',
  'entities': {'url': {'urls': [{'url': 'http://t.co/xQ

In [9]:
#get values lists
time = []
twtid = []
text = []
favorite_count = []
retweet_count=[]
hashtags=[]
url=[]

favorite_count=[]
for line in txts:
    time.append(line['created_at'])
    twtid.append(line['id'])
    text.append(line['text'])
    favorite_count.append(line['favorite_count'])
    retweet_count.append(line['retweet_count'])
    hashtags.append(line['entities']['hashtags'])
    url.append(line['entities']['urls'])

In [10]:
pip install preprocessing

Note: you may need to restart the kernel to use updated packages.


In [11]:
!pip install tweet-preprocessor

In [12]:
# remove urls and hashtags in text
#!pip install tweet-preprocessor 

import preprocessor as p
p.clean(text[1])

# remove hushtags and urls in all descriptions
text_clean = []
for line in text:
    text_clean.append(p.clean(line))
text_clean[:1]

["Happy . Here's how we're celebrating:"]

# 2. Assessing Data

In [13]:
# create data frame
df = pd.DataFrame(list(zip(time, twtid, text, favorite_count, retweet_count, hashtags, url)),
                             columns=['created_at', 'tweet_id','text_clean','favorite_count','retweet_count',
                                     'hashtags','url'])
df

,created_at,tweet_id,text_clean,favorite_count,retweet_count,hashtags,url
0,Fri Jul 31 17:03:42 +0000 2020,1289245366081093632,Happy #NationalAvocadoDay. 🥑 Here's how we're ...,6,1,"[{'text': 'NationalAvocadoDay', 'indices': [6,...","[{'url': 'https://t.co/gE60WESbnB', 'expanded_..."
1,Fri Jul 31 15:03:04 +0000 2020,1289215003954831360,Before figuring out how your own body reacts t...,8,3,[],"[{'url': 'https://t.co/ixbJop7ZvT', 'expanded_..."
2,Fri Jul 31 14:03:14 +0000 2020,1289199948404101120,"We caught up with functional medicine leaders,...",7,3,[],"[{'url': 'https://t.co/atohj6rQy9', 'expanded_..."
3,Fri Jul 31 13:02:53 +0000 2020,1289184761349459969,To acknowledge the beginning of harvest season...,10,3,[],"[{'url': 'https://t.co/jceakbNXM7', 'expanded_..."
4,Fri Jul 31 11:02:15 +0000 2020,1289154401253371904,You'll get the *hang* of these at-home moves i...,4,0,[],"[{'url': 'https://t.co/Ceyp3fOLl5', 'expanded_..."
...,...,...,...,...,...,...,...
3408,Thu Nov 28 06:00:19 +0000 2019,1199931001599860736,"""I fell in love with the texture at first use,...",8,2,[],"[{'url': 'https://t.co/NhvEuTumKX', 'expanded_..."
3409,Thu Nov 28 04:00:10 +0000 2019,1199900768125562880,The 3 common ways your shadow may be sabotagin...,2,0,[],"[{'url': 'https://t.co/z7FxuXG1nX', 'expanded_..."
3410,Thu Nov 28 02:00:35 +0000 2019,1199870672077500416,Our love affair with all things tech is taking...,13,11,[],"[{'url': 'https://t.co/4nJUUbjAbm', 'expanded_..."
3411,Thu Nov 28 00:00:54 +0000 2019,1199840553552232450,Here are both preventive strategies this exper...,15,7,[],"[{'url': 'https://t.co/c97nBD2MeC', 'expanded_..."


In [14]:
df['url'].value_counts()

[]                                                                                                                                                                                   14
[{'url': 'https://t.co/pkHBq30Na3', 'expanded_url': 'http://ow.ly/LdSB30r0pJg', 'display_url': 'ow.ly/LdSB30r0pJg', 'indices': [86, 109]}]                                            2
[{'url': 'https://t.co/ZFikjGeoni', 'expanded_url': 'https://twitter.com/i/web/status/1287523247320432642', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]     2
[{'url': 'https://t.co/5lSkhZ1dir', 'expanded_url': 'https://twitter.com/i/web/status/1287432927391907844', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}]     2
[{'url': 'https://t.co/mDJEEOREFe', 'expanded_url': 'http://ow.ly/w2SE30qZeKQ', 'display_url': 'ow.ly/w2SE30qZeKQ', 'indices': [103, 126]}]                                           2
                                                                                

In [15]:
df['text_clean'].value_counts()

This lesser-known form of fasting takes it a step further by making sure meals are in line with your body's interna… https://t.co/YEqPFCKvaZ    2
Oils are the workhorses of natural + clean hair care. 💧 And this one stands out amongst the best of them. https://t.co/buYyfaFCjT               2
A link between olfaction and dementia? 👃 We're *sniffing* out this new research. https://t.co/3IrDyzLYsX                                        2
We got the experts to break down the pros and cons of both. 🧼 https://t.co/DMDO2mRqO6                                                           2
Bonus: It's vegetarian. 🥗 https://t.co/0PocHqsIlW                                                                                               2
                                                                                                                                               ..
Is this truly nature's *do*-it-all ingredient? 😏 https://t.co/LLtFML0OXS                                                    

In [16]:
df.describe()

,tweet_id,favorite_count,retweet_count
count,3.413000e+03,3413.000000,3413.000000
mean,1.246019e+18,10.918840,3.947260
std,2.685540e+16,8.122424,3.348921
min,1.199810e+18,0.000000,0.000000
25%,1.222701e+18,5.000000,2.000000
50%,1.245728e+18,9.000000,3.000000
75%,1.269676e+18,14.000000,5.000000
max,1.289245e+18,73.000000,28.000000


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3413 entries, 0 to 3412
Data columns (total 7 columns):
created_at        3413 non-null object
tweet_id          3413 non-null int64
text_clean        3413 non-null object
favorite_count    3413 non-null int64
retweet_count     3413 non-null int64
hashtags          3413 non-null object
url               3413 non-null object
dtypes: int64(3), object(4)
memory usage: 186.8+ KB


# 3. Clean & Store Data

In [18]:
#copy data
clean_df= df.copy()

In [19]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3413 entries, 0 to 3412
Data columns (total 7 columns):
created_at        3413 non-null object
tweet_id          3413 non-null int64
text_clean        3413 non-null object
favorite_count    3413 non-null int64
retweet_count     3413 non-null int64
hashtags          3413 non-null object
url               3413 non-null object
dtypes: int64(3), object(4)
memory usage: 186.8+ KB


CLEAN HASHTAGS

In [20]:
clean_df['hashtags'].value_counts()

[]                                                           3301
[{'text': 'mbgPodcast', 'indices': [8, 19]}]                   25
[{'text': 'mbgPodcast', 'indices': [11, 22]}]                  14
[{'text': 'mbgpartner', 'indices': [127, 138]}]                 2
[{'text': 'NationalIceCreamDay', 'indices': [5, 25]}]           2
                                                             ... 
[{'text': 'NationalDisabilityDay', 'indices': [14, 36]}]        1
[{'text': 'mbgpartner', 'indices': [117, 128]}]                 1
[{'text': 'InternationalWomensDay', 'indices': [26, 49]}]       1
[{'text': 'mbgPodcast', 'indices': [56, 67]}]                   1
[{'text': 'mbgPodcast', 'indices': [70, 81]}]                   1
Name: hashtags, Length: 67, dtype: int64

In [21]:
import re
import string

In [22]:
#remove '[]' in hashtags
clean_df.hashtags = clean_df.hashtags.replace('[]','')

In [23]:
#convert hashtags into string
clean_df['hashtags']=clean_df['hashtags'].apply(str)

In [24]:
# look behind
re.findall(r"(?<='text':\s\')\w+",clean_df.hashtags[3])

[]

In [25]:
' '.join(re.findall(r"(?<='text':\s\')\w+",clean_df.hashtags[3]))

''

In [26]:
# apply to every rows
clean_df.hashtags=clean_df.hashtags.apply(lambda row: ' '.join(re.findall(r"(?<='text':\s\')\w+",row)))

In [27]:
#check again
clean_df.hashtags.value_counts()

                                      3301
mbgPodcast                              54
mbgpartner                              16
TraderJoes                               8
NationalIceCreamDay                      4
NationalHempDay                          3
NationalAvocadoDay                       2
AmericanPistachios mbgpartner            2
mbgpodcast                               1
BettyWhite                               1
InternationalWomensDay                   1
EarthDay                                 1
aerieReal                                1
ad                                       1
PlasticFreeJuly                          1
Clean15                                  1
EllieKemper                              1
chomps cantstopwontstop mbgpartner       1
mbgPodcasat                              1
BlackBotanistsWeek                       1
NationalDisabilityDay                    1
covergirls                               1
ReneeZellweger                           1
GalentinesD

CONVERT TIME TO RIGHT DATA TYPE & DATETIME

In [28]:
# Convert timestamp datatype to datetime
clean_df['created_at'] = pd.to_datetime(clean_df['created_at'], infer_datetime_format=True)

In [32]:
#Check again
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3413 entries, 0 to 3412
Data columns (total 7 columns):
created_at        3413 non-null datetime64[ns, UTC]
tweet_id          3413 non-null int64
text_clean        3413 non-null object
favorite_count    3413 non-null int64
retweet_count     3413 non-null int64
hashtags          3413 non-null object
url               3413 non-null object
dtypes: datetime64[ns, UTC](1), int64(3), object(3)
memory usage: 186.8+ KB


In [33]:
clean_df.to_csv('Clean_data_Mindbodygreen.csv', encoding='utf-8', index=False)

In [34]:
#CHECK AGAIN
new_df = pd.read_csv('Clean_data_Mindbodygreen.csv')
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3413 entries, 0 to 3412
Data columns (total 7 columns):
created_at        3413 non-null object
tweet_id          3413 non-null int64
text_clean        3413 non-null object
favorite_count    3413 non-null int64
retweet_count     3413 non-null int64
hashtags          112 non-null object
url               3413 non-null object
dtypes: int64(3), object(4)
memory usage: 186.8+ KB
